In [1]:
import numpy as np
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt
import keras
from shutil import copyfile
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
os.makedirs('train_new/cats')
os.makedirs('train_new/dogs')
os.makedirs('validation_new/cats')
os.makedirs('validation_new/dogs')

In [3]:
dogs = [filename for filename in os.listdir('train1') if filename.startswith("dog")]

In [4]:
cats = [filename for filename in os.listdir('train1') if filename.startswith("cat")]

In [5]:
for i in range(1000):
    copyfile('train1/' + dogs[i], 'train_new/dogs/' + dogs[i])

In [6]:
for i in range(1000):
    copyfile('train1/' + cats[i], 'train_new/cats/' + cats[i])

In [7]:
for i in range(1000, 1400):
    copyfile('train1/' + dogs[i], 'validation_new/dogs/' + dogs[i])

In [8]:
for i in range(1000, 1400):
    copyfile('train1/' + cats[i], 'validation_new/cats/' + cats[i])


In [9]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'train_new'
validation_data_dir = 'validation_new'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 88s 706ms/step - loss: 0.7137 - acc: 0.5040 - val_loss: 0.6806 - val_acc: 0.6038
Epoch 2/50
125/125 [==============================] - 86s 690ms/step - loss: 0.6871 - acc: 0.5680 - val_loss: 0.6386 - val_acc: 0.6112
Epoch 3/50
125/125 [==============================] - 86s 691ms/step - loss: 0.6556 - acc: 0.6350 - val_loss: 0.6092 - val_acc: 0.6675
Epoch 4/50
125/125 [==============================] - 88s 700ms/step - loss: 0.6393 - acc: 0.6530 - val_loss: 0.8316 - val_acc: 0.6012
Epoch 5/50
125/125 [==============================] - 89s 710ms/step - loss: 0.6092 - acc: 0.6785 - val_loss: 0.5998 - val_acc: 0.6750
Epoch 6/50
125/125 [==============================] - 91s 730ms/step - loss: 0.6020 - acc: 0.6995 - val_loss: 0.5917 - val_acc: 0.7025
Epoch 7/50
125/125 [==============================] - 89s 716ms/step - loss: 0.5880 - acc: 0.6930 - val_los